# Political Party Classifier Using Tweets  
**Author**: Anael Umar  
**GitHub:** [Anael-UmarDS](https://github.com/Anael-UmarDS)

---

# Description

This project classifies tweets written by US politicians as either **Democratic** or **Republican**, using a **supervised machine learning approach**. Tweets are processed with **TF-IDF vectorization**, and a **Logistic Regression classifier** is trained to predict the political party based solely on the tweet's content. This project demonstrates real-world **natural language processing (NLP)**, **feature engineering**, and **binary classification**, applied to political data.

---

# Objectives

- Use **supervised learning** to **classify** political tweets into Democratic or Republican categories  
- Understand how **political language differs by party affiliation**
- Build a clean, interpretable **machine learning pipeline** using scikit-learn  
- **Apply NLP** techniques on real-world, publicly available tweet data  
- Develop skills in **preprocessing, feature extraction, and model evaluation**
- Explores how **data science** intersects with **government and civic technology**

---

# Source of Datasets

[Kaggle Dataset for **dataset.csv**](https://www.kaggle.com/datasets/mrmorj/us-politicians-twitter-dataset)

[Kaggle Dataset for **us_tweets.csv**](https://www.kaggle.com/datasets/jeevanbhoot/tweets-from-us-politicians)

---
# How It Works

### Data Preparation  
- Tweets were gathered from U.S. politicians, merged with a dataset containing each user's party affiliation  
- Only tweets from **Democratic Party** and **Republican Party** members were retained  
- Merged dataset included:  
  - `Name`, `Twitter_username`, `Political_party`, and `text`

### Text Preprocessing  
Each tweet undergoes data cleaning:
- Lowercasing  
- Removing URLs, mentions, hashtags, and punctuation  
- Removing common stopwords  
- Final cleaned text is stored for vectorization

### Feature Extraction (TF-IDF)  
- **TfidfVectorizer** converts each cleaned tweet into a **feature vector**
- Captures the **importance** of each word **relative** to the **entire dataset**

### Model Training (Logistic Regression)  
- Binary labels:
  - **Democratic Party**
  - **Republican Party**
- Model is trained using **LogisticRegression** from scikit-learn  

---

# Example Use Cases

- **Civic Tech**: Understand how politicians from different parties frame similar topics  
- **Political Science Research**: Study the linguistic differences in party messaging  
- **Media Literacy**: Recognize bias through language cues

---

# Visualizations

- **Confusion Matrix** for evaluating prediction accuracy  
- **Word importance** (TF-IDF weights) by class
- **Sample predictions** of tweet text with predicted party label  

---

# Limitations

- **Dataset Bias**: Only includes verified U.S. politicians; may not generalize to regular users  
- **Context Missing**: A single tweet may not represent a user's overall ideology  
- **Imbalanced Data**: Class imbalance can affect generalization (requires resampling if necessary)  
- **Linguistic Ambiguity**: Some tweets may appear neutral but still hold ideological undertones  

---

# Importing Libraries

In [22]:
# Import required libraries

import pandas as pd                                                                             # Used for data manipulation and analysis
import numpy as np                                                                              # Used to store data in multi-dimensional arrays
import matplotlib.pyplot as plt                                                                 # Used to illustrate data as plots or graphs
import joblib                                                                                   # Used to run large computations and efficiently handle them
from collections import Counter                                                                 # Used to iterate through hashable objects
from typing import List                                                                         # Used to indicate the expected types of variables, function parameters, and return values
from nltk.sentiment import SentimentIntensityAnalyzer                                           # Give a sentiment intensity score to sentences
from sklearn.feature_extraction.text import TfidfVectorizer                                     # Convert a collection of raw documents to a matrix of TF-IDF features
from sklearn.linear_model import LogisticRegression                                             # Used for multi-class classification problems
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix             # Used for the accuracy of a classification model, a text report showing the main classification metrics for each class, computing a confusion matrix, respectively
from sklearn.model_selection import train_test_split                                            # Split arrays or matrices into random train and test subsets
import nltk                                                                                     # Used for working with Natural Language Processing (NLP)
from nltk.corpus import stopwords                                                               # Used for identifying and removing stopwords
from textblob import TextBlob                                                                   # Used for processing textual data
import re                                                                                       # Used for regular expression

nltk.download("stopwords")                                                                      # Using NLTK Library to download stopwords, filler words like "the", "a", etc.
stop_words = set(stopwords.words("english"))                                                    # Set language to english

# Merging Relevant Data

In [19]:
dataset = pd.read_csv("dataset.csv")
#dataset.head()

In [20]:
us_tweets = pd.read_csv("us_tweets.csv")
#us_tweets.head()

In [4]:
# Filtering data for the two-party system
df = dataset[(dataset["Political_party"]=="Democratic Party") | (dataset["Political_party"]=="Republican Party")].copy()

In [5]:
cols = ["Name", "Twitter_username","Political_party"]
df = df[cols]
df = df.dropna(subset=["Name", "Twitter_username", "Political_party"])

In [6]:
text_column = us_tweets["text"]

In [7]:
merged_df = pd.merge(df, us_tweets, on="Twitter_username", how="inner")

cols = ["Name", "Twitter_username", "text", "Political_party"]

merged_df = merged_df[cols]
merged_df = merged_df.dropna(subset=cols)

In [8]:
df = merged_df
df = df.copy()
# df.head(n=41) shows us that every 40 rows is dedicated to one policymaker
df.head()

,Name,Twitter_username,text,Political_party
0,A. Donald McEachin,RepMcEachin,It is truly heartbreaking to learn of the deat...,Democratic Party
1,A. Donald McEachin,RepMcEachin,I am joining my colleagues @davidcicilline @Re...,Democratic Party
2,A. Donald McEachin,RepMcEachin,Yesterday a mob of insurrectionists breached t...,Democratic Party
3,A. Donald McEachin,RepMcEachin,Today’s violent riot at the U.S. Capitol was a...,Democratic Party
4,A. Donald McEachin,RepMcEachin,Either President Trump needs to be immediately...,Democratic Party


# Cleaning and Preparing Data for Training and Evaluation

In [9]:
# Clean data to lowercase and no blanks
# Helper function

def basic_text_clean(text: str) -> str:

    text = text.lower().strip()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)

    # Remove special characters and punctuation, keeping only alphanumeric and spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    words = []
    for word in text.split():
      if word.isalpha() and word not in stop_words:
        words.append(word)

    return ' '.join(words)

In [10]:
# Clean Pandas DataFrame using helper function

def prepare_features(df: pd.DataFrame):
    raw_text = df["text"].apply(basic_text_clean)
    label = df["Political_party"]
    return raw_text, label

# Using a Training Model to Classify Ideology Based on Text and Political Party

In [11]:
# Labels for classification

LABELS = ["Democratic Party", "Republican Party"]

In [12]:
# Training function to train the model to classify tweets
# TF-IDF (Term Frequency - Inverse Document Frequency)
# TF: Term Frequency: number of times the term appears in a document compared to the total number of words in the document
# IDF: Inverse Document Frequency: ratio of total number of documents to number of documents containing the term
# Uses TF-IDF Sentiment Weightage System: TF-IDF = TF * IDF
# Low TF-IDF indicates a high frequency of a term
# Logistic regression: statistical analysis method used to predict a binomial outcome

def train_model(raw_text, label):

  vectorizer = TfidfVectorizer(max_features=6000, ngram_range=(1, 2))                                           # Tokenize the text into numerical features
  raw_text_vec = vectorizer.fit_transform(raw_text)                                                             # Fit the vectorizer to raw_text and transform it (evaluating how important the word is to the document)
  model = LogisticRegression(max_iter=2000, n_jobs=-1, class_weight="balanced", random_state=42)                # Initialize a Logistic Regression model for classification

  model.fit(raw_text_vec, label)                                                                                # Train the Logistic Regression model using the vectorized training features for raw_text and their corresponding labels of Political_party
  return model, vectorizer                                                                                      # Return trained model and numerical features

In [13]:
# Predicts party for each tweet based on TF-IDF weight

def predict(model, vectorizer, political_party: List[str]):
    cleaned = [basic_text_clean(tweet) for tweet in political_party]
    return model.predict(vectorizer.transform(cleaned))

In [14]:
# Evaluation of the model by calculating the accuracy of predictions and calling plot_confusion_matrix
# Calculates precision, recall, and trade-off rate of precision and recall (f1-score)

def evaluate_model(model, vectorizer, X_test, y_test):
    X_test_vec = vectorizer.transform(X_test)
    y_pred = model.predict(X_test_vec)
    print("\nEvaluation:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}\n")
    print(classification_report(y_test, y_pred, digits=3))
    cm = confusion_matrix(y_test, y_pred, labels=LABELS)
    plot_confusion_matrix(cm)

# Visualization

In [15]:
# Plots confusion matrix that compares true labels vs assigned labels
# And saves as an image called "confusion_matrix.png"

def plot_confusion_matrix(cm: np.ndarray):
    fig, ax = plt.subplots(figsize=(6, 4))
    im = ax.imshow(cm, interpolation="nearest", cmap="Blues")
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(len(LABELS)),
           yticks=np.arange(len(LABELS)),
           xticklabels=LABELS,
           yticklabels=LABELS,
           ylabel="True label",
           xlabel="Predicted label",
           title="Confusion Matrix")
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], "d"), ha="center", va="center", color="black")
    fig.tight_layout()
    plt.savefig("confusion_matrix.png", dpi=300)
    plt.close()
    print("Confusion matrix saved → confusion_matrix.png")

In [16]:
# Efficiently save the models using joblib

def save_artifacts(model, vectorizer):
    joblib.dump(model, "text_model.joblib")
    joblib.dump(vectorizer, "tfidf_vectorizer.joblib")
    print("Model artifacts saved")

# Main Function

In [17]:
# Main Running Function

def main():

  # Print how many tweets are of each of the two major political party labels
  print("Class distribution:", dict(Counter(df["Political_party"])))

  # Clean data and prepare for training
  # Implement stratified random sampling
  # Stratified random sampling splits data into subsets
  # and randomly chooses each from there until None are left

  X, Y = prepare_features(df)
  min_class_size = min(Counter(Y).values())
  can_stratify = min_class_size >= 2

  X_train, X_test, Y_train, Y_test = train_test_split(
      X, Y, test_size=0.1, stratify=Y if can_stratify else None, random_state=42
  )

  # Train model (see function documentation comments on how)
  model, vectorizer = train_model(X_train, Y_train)

  # Evaluate model (see function documentation comments on how)
  evaluate_model(model, vectorizer, X_test, Y_test)

  # Save models using save_artifact (see function documentation comments on how)
  save_artifacts(model, vectorizer)

  # List of fake demo tweets randomly generated by Google Gemini
  demo_tweets = [
    "Biden's spending spree is driving inflation and hurting American families. We need fiscal responsibility and policies that empower businesses, not burden them with regulations. #CutSpending #LowerTaxes",
    "Standing with working families tonight. We need to raise the minimum wage, protect collective bargaining rights, and build an economy that works for everyone, not just the wealthy few. #WorkersRights #FairWage",
    "Our border is in crisis. We must secure our sovereignty, support our brave border patrol agents, and enforce our laws to protect our nation. #SecureOurBorders #NationalSecurity",
    "Honored to meet with brave law enforcement officers today. We must always support those who put their lives on the line to keep our communities safe. Back the Blue! #ThinBlueLine #SupportOurPolice",
    "The American dream is built on hard work and opportunity. We need to unleash the power of the free market and empower entrepreneurs to create jobs and prosperity for all. #FreeMarket #AmericanDream",
    "Proud to stand for individual liberty and personal responsibility. Government overreach stifles innovation and limits the freedom that makes America exceptional. #Freedom #LimitedGovernment",
    "Climate change is an existential threat, and we can't afford to wait. Investing in renewable energy creates jobs and protects our planet for future generations. #GreenEconomy #ActOnClimate",
    "Had a productive discussion with local educators today. We need to fully fund our public schools and ensure every student has the resources they need to succeed, regardless of zip code. #InvestInEducation #PublicSchools",
    "Every family deserves access to affordable healthcare. It's not a privilege, it's a right. We must strengthen the ACA and expand coverage for all Americans. #HealthcareForAll #ACA",
    "It's time to address systemic inequities in our justice system. We need comprehensive police reform and a focus on community-based solutions to build trust and ensure justice for all. #JusticeReform #Equality"
  ]

  # Print Results
  print("\nDemo predictions:")
  for tweet, party in zip(demo_tweets, predict(model, vectorizer, demo_tweets)):
      print(f"[{party}] {tweet}")

In [21]:
# Run main()

if __name__ == "__main__":
  main()

# Conclusions
- **TF-IDF + Logistic Regression** performs well in classifying political party from tweet content
- Simple **NLP methods** can reveal **deep political divisions** in language use
- **Machine learning** can support **political discourse analysis, civic education, and transparency**

# Implications
### Government & Civics
- Understand how **partisan messagin**g is framed
- Provide **transparency** in political communication

### Political Research
- **Track** evolving **language by party** over time
- Study **messaging strategy** and framing during election campaigns

### AI & ML Development
- Uses explainable **machine learning models** for interpretable **results**

# Machine Learning Concepts
- Supervised Learning
- Binary Classification
- Text Cleaning & Preprocessing
- TF-IDF Feature Engineering
- Logistic Regression
- Label Encoding
- Evaluation Metrics